<a href="https://www.inove.com.ar"><img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/PA%20Banner.png" width="1000" align="center"></a>


# Ejercicio de clasificación con redes neuronales convolucionales (CNN)

Ejemplo de clasificación utilizando redes neuronales para la clasificación de imagenes<br>

v1.1

In [ ]:
import os
import platform

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import keras
from keras.models import Sequential
#from keras.utils import to_categorical
from keras.utils.np_utils import to_categorical # Si esto no funciona, probar con el import anterior

# Recolectar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline1.png" width="1000" align="middle">

In [ ]:
from keras.datasets import mnist

# Leer el dataset de mnist
(data_X_train, data_y_train), (data_X_test, data_y_test) = mnist.load_data()

### `MNIST dataset`:
El dataset **`MNIST`** contiene 70.000 imagenes de números escritos a mano (números del 0 al 9, 10 dígitos). Cada imagen es de 28x28 píxeles en escala de grises (1 canal o 1 nivel de profundidad). Es uno de los dataset más utilizados para poner a prueba algoritmos de clasificación de imagenes.<br> [Dataset source](https://keras.io/api/datasets/mnist/)
- La entrada (X) es una variable imagen de 28x28
- La salida (y) es el dígito que representa la imagen en cuestión, un número de 0 al 9


In [ ]:
# Visualizar las 100 primeras imagenes
fig = plt.figure(figsize=(16,9))
for i in range(100):
    ax = fig.add_subplot(10, 10, i+1)
    ax.axis('off')
    plt.imshow(data_X_train[i], cmap='Greys')
plt.show()

In [ ]:
plt.imshow(data_X_train[0], cmap='gray')
plt.title("Número: " + str(data_y_train[0]))
plt.show()

# Procesar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline2.png" width="1000" align="middle">

In [ ]:
# Observar como está representada la imagen, ver fila del medio (14)
print(data_X_train[0][14, :])

In [ ]:
# Por los resultados podemos ver que la imagen está representada de 0 a 255
# Normalizamos los datos para que se encuentren entre 0 y 1
X_train_norm = data_X_train / 255
X_test_norm = data_X_test / 255

In [ ]:
print('Cantidad de datos en observacion:', X_train_norm.shape[0])

In [ ]:
print('Tamaño de la imagen:', X_train_norm[0].shape)

In [ ]:
# En estas imagenes si nescalas de grieses no vienen con el canal especificado
# Deberia ser (28, 28, 1) para poder utilizar en una red CNN
# Agregar el [samples][pixels][width][height]
X_train = X_train_norm.reshape(data_X_train.shape[0], data_X_train.shape[1], data_X_train.shape[2], 1).astype('float32')
X_test = X_test_norm.reshape(data_X_test.shape[0], data_X_test.shape[1], data_X_test.shape[2], 1).astype('float32')
X_train.shape

# Explorar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline3.png" width="1000" align="middle">

In [ ]:
# Observar el los primeros 10 datos del dataset de y_train
print(data_y_train[:10])

In [ ]:
# Exploramos los primeros diez 8 del dataset de train
# Para ser graficando en escala de grises utilizamos el dataset
# normalizado sin especificar la cantidad de canales
fig = plt.figure(figsize=(16,9))
j = 0

for i in range(10):
    ax = fig.add_subplot(1, 10, i+1)
    ax.axis('off')
    while True:
        if data_y_train[j] == 8:
            ax.imshow(X_train_norm[j], cmap='Greys')
            j += 1
            break
        j += 1
plt.show()

In [ ]:
# Exploramos los primeros diez 8 del dataset de test
# Para ser graficando en escala de grises utilizamos el dataset
# normalizado sin especificar la cantidad de canales
fig = plt.figure(figsize=(16,9))
j = 0

for i in range(10):
    ax = fig.add_subplot(1, 10, i+1)
    ax.axis('off')
    while True:
        if data_y_test[j] == 8:
            ax.imshow(X_test_norm[j], cmap='Greys')
            j += 1
            break
        j += 1
plt.show()

# Entrenar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline4.png" width="1000" align="middle">

Los datos ya estan dividios en train y test

In [ ]:
# Transformar la salida a oneHotEncoding con to_categorical
y_train = to_categorical(data_y_train)
y_test = to_categorical(data_y_test)
y_train[:10]

In [ ]:
# input shape
in_shape = X_train.shape[1:]
in_shape

In [ ]:
# output shape
out_shape = y_train.shape[1]
out_shape

In [ ]:
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D

model = Sequential()

# Se utiliza 8 filtros de entrada y una sola par de capas CONV + POOL
# para tener la misma cantidad de parámetros que los utilizados
# en la redes ANN y DNN

model.add(Conv2D(filters=8, kernel_size=(5, 5), strides=(1, 1), padding='same', activation='relu', input_shape=in_shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Flatten())
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=out_shape, activation='softmax'))

model.compile(optimizer="Adam",
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
history = model.fit(X_train, y_train, validation_split=0.2 , epochs=10, batch_size=128)

In [ ]:
epoch_count = range(1, len(history.history['accuracy']) + 1)
sns.lineplot(x=epoch_count,  y=history.history['accuracy'], label='train')
sns.lineplot(x=epoch_count,  y=history.history['val_accuracy'], label='valid')
plt.show()

In [ ]:
y_hat_prob = model.predict(X_test)
y_hat_prob[:3]

In [ ]:
y_hat = np.argmax(y_hat_prob,axis=1)
y_hat[:3]

# Validar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline5.png" width="1000" align="middle">

In [ ]:
# Calcular la exactitud (accuracy)
scores = model.evaluate(X_test, y_test)
scores[1]

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(y_test.argmax(axis=1), y_hat)
cmd = ConfusionMatrixDisplay(cm, display_labels=list(range(10)))
cmd.plot(cmap=plt.cm.Blues)
plt.show()

# Utilizar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline6.png" width="1000" align="middle">

Se utiliza el ranking de los peores 10 números clasificados con una ANN para evlauar contra este nuevo modelo de red neuronal

In [ ]:
ranking_10 = [8183, 6765, 8522, 1325, 582, 9280, 5749, 3567, 3206, 9744]

In [ ]:
# Obtener los vectores para evaluar
X_test_peores = X_test[ranking_10]
y_test_peores = y_test[ranking_10]

In [ ]:
# Calcular la exactitud
score = model.evaluate(X_test_peores, y_test_peores)
score[1]

In [ ]:
# ¿Qué es lo que el sistema ve?
y_hat_prob_peores = model.predict(X_test_peores)
y_hat_peores = np.argmax(y_hat_prob_peores,axis=1)
y_hat_peores

In [ ]:

fig = plt.figure(figsize=(16,9))
j = 0

for i in ranking_10:
    ax = fig.add_subplot(1, 10, j+1)
    ax.axis('off')
    ax.imshow(X_test_norm[i], cmap='Greys')
    ax.set_title("Número: " + str(y_hat_peores[j]))
    j += 1

plt.show()

# Conclusión
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline7.png" width="1000" align="middle">

Al utilizar una red convolucional se obtuvo un resultado mucho mejor que en mismos ejemplos con ANN y DNN (deep larning). Esta red pudo caracterizar mejor los dígitos del dataset.

# Ejemplo con Radio

Ejemplo con Gradio
Gradio es una librería para realizar GUI rápida de prueba para nuestros modelos\ 
Referencias:

- https://www.gradio.app/getting_started
- https://www.gradio.app/
- https://www.gradio.app/docs

In [ ]:
# Instalar Gradio
import sys
!{sys.executable} -m pip install gradio --quiet

In [ ]:
import gradio as gr

def recognize_digit(img):
    # Normalizacion 0..1
    img = img / 255.0

    # Flatten
    num_pixels = img.shape[0] * img.shape[1]
    img = img.reshape(1, num_pixels).astype('float32')

    # Predict
    prediction = model.predict([img]).tolist()[0]
    # Mostrar el puntaje de prediccion para cada digito
    return {str(i): prediction[i] for i in range(10)}

iface = gr.Interface(fn=recognize_digit, inputs="sketchpad", outputs="label").launch(debug=True)